<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Neuronale Netze - Keras Regression - Diamonds
</b></font> </br></p>

---


In [ ]:
#@title ðŸ”§ Colab-Umgebung { display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from ml_lib.utilities import get_ipinfo
import sys
print()
print(f"Python Version: {sys.version}")
print()
get_ipinfo()

# 0  | Install & Import
***

In [ ]:
# Install

In [ ]:
# Import
from pandas import read_csv, DataFrame, concat
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

from keras.layers import Input, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

import plotly.express as px
import plotly.subplots as sp

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---


<p><font color='black' size="5">ðŸ“‹ Checkliste</font></p>

âœ… Aufgabe verstehen</br>
âœ… Daten sammeln</br>
âœ… Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
âœ… Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
âœ… Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---

Dieser klassische Datensatz enthÃ¤lt die Preise und andere Attribute von fast 54.000 Diamanten.



[DataSet](https://www.openml.org/search?type=data&status=active&id=42225)

[Info](https://www.kaggle.com/datasets/shivam2503/diamonds)


In [ ]:
df = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02%20data/diamonds.csv",
    usecols=[
        "carat",
        "cut",
        "color",
        "clarity",
        "depth",
        "table",
        "price",
    ],
)

In [ ]:
data = df.copy()
target = data.pop("price")

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.groupby("cut").count()

In [ ]:
data.groupby("color").count()

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Plotly
</font></p>

In [ ]:
title_ = "Depth"
b1 = px.box(data["depth"], title=title_, width=600, height=600)

title_ = "Carat"
b2 = px.box(data["carat"], title=title_, width=600, height=600)

title_ = "Table"
b3 = px.box(data["table"], title=title_, width=600, height=600)

fig = sp.make_subplots(rows=1, cols=3, subplot_titles=("Depth", "Carat", "Table"))

for trace in b1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in b2.data:
    fig.add_trace(trace, row=1, col=2)

for trace in b3.data:
    fig.add_trace(trace, row=1, col=3)

# Layout anpassen
fig.update_layout(width=1000, height=500, title_text="Box-Plots")

# Plot anzeigen
fig.show()

# 2 |  Prepare

---

<p><font color='black' size="5">ðŸ“‹ Checkliste</font></p>

âœ… Nicht benÃ¶tigte Features lÃ¶schen</br>
âœ… Datentyp ermitteln/Ã¤ndern</br>
âœ… Duplikate ermitteln/lÃ¶schen</br>
âœ… Missing Values behandeln</br>
âœ… AusreiÃŸer behandeln</br>
âœ… Kategorischer Features Kodieren</br>
âœ… Numerischer Features skalieren</br>
âœ… Feature-Engineering (neue Features schaffen)</br>
âœ… DimensionalitÃ¤t reduzieren</br>
âœ… Resampling (Over-/Undersampling)</br>
âœ… Pipeline erstellen/konfigurieren</br>
âœ… Train-Test-Split durchfÃ¼hren</br>

<p><font color='black' size="5">
Datentyp ermitteln
</font></p>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<p><font color='black' size="5">
Kodierung
</font></p>

In [ ]:
cat_seq = [
    ["Fair", "Good", "Very Good", "Premium", "Ideal"],
    ["J", "I", "H", "G", "F", "E", "D"],
    ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"],
]
coder = OrdinalEncoder(categories=cat_seq)
data[cat_col] = coder.fit_transform(data[cat_col])

<p><font color='black' size="5">
Skalierung
</font></p>

In [ ]:
scaler = MinMaxScaler()
data[num_col] = scaler.fit_transform(data[num_col])


<p><font color='black' size="5">
Train-Test-Split
</font></p>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.3, random_state=42
)

# 3 | Modeling
---

<p><font color='black' size="5">ðŸ“‹ Checkliste</font></p>

âœ… Modellauswahl treffen</br>
âœ… Pipeline erweitern/konfigurieren</br>
âœ… Training durchfÃ¼hren</br>
âœ… Hyperparameter Tuning</br>
âœ… Cross-Valdiation</br>
âœ… Bootstrapping</br>
âœ… Regularization</br>

<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font color='black' size="5">
Aufbau des Neuronale Netzes
</font></p>


In [ ]:
model = Sequential()
model.add(Input(shape=(6,)))
model.add(Dense(units=88, activation="relu"))
model.add(Dense(units=44, activation="relu"))
model.add(Dense(units=11, activation="relu"))
model.add(Dense(1))

In [ ]:
# Visualisierung neuronales Netz
model.summary()

In [ ]:
# Visualisierung neuronales Netz
plot_model(
    model,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in model.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")

<p><font color='black' size="5">
Compile
</font></p>



In [ ]:
model.compile(optimizer="adam", loss="mae")

<p><font color='black' size="5">
Callback
</font></p>

In [ ]:
# early = EarlyStopping(monitor="loss", patience=5)

# reduce_lr = ReduceLROnPlateau(
#     monitor="loss",
#     factor=0.1,
#     patience=2,
#     verbose=1,
#     mode="auto",
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=0,
# )

# check = ModelCheckpoint(filepath="model.keras", monitor="loss", save_best_only=True)

<p><font color='black' size="5">
Training
</font></p>

In [ ]:
model.fit(
    data_train,
    target_train,
    epochs=20,
    validation_split=0.2,
    # callbacks=[early, reduce_lr, check],
)

In [ ]:
print(model.history.params)
print(model.history.history.keys())
save_history = model.history.history.keys()

<p><font color='black' size="5">
Loss-Entwickung
</font></p>

In [ ]:
title_ = "Loss-Entwicklung"
px.line(
    y=model.history.history["loss"],
    title=title_,
    labels={"x": "Epochen", "y": "Loss-Wert"},
    width=800,
    height=400,
)

# 4 | Evaluate
---

<p><font color='black' size="5">ðŸ“‹ Checkliste</font></p>

âœ… Prognose (Train, Test) erstellen</br>
âœ… ModellgÃ¼te prÃ¼fen</br>
âœ… Residuenanalyse erstellen</br>
âœ… Feature Importance/Selektion prÃ¼fen</br>
âœ… Robustheitstest erstellen</br>
âœ… Modellinterpretation erstellen</br>
âœ… SensitivitÃ¤tsanalyse erstellen</br>
âœ… Kommunikation (Key Takeaways)</br>


<p><font color='black' size="5">
Prediction
</font></p>


In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
Bestimmtheitsmass
</font></p>

In [ ]:
r2 = r2_score(target_train, target_train_pred)
print(f"Modell: {model} -- Train --- Bestimmtheitsmass: {r2:5.2f}")

In [ ]:
r2 = r2_score(target_test, target_test_pred)
print(f"Modell: {model} -- Test --- Bestimmtheitsmass: {r2:5.2f}")

<p><font color='black' size="5">
Mean Absolut Error
</font></p>

In [ ]:
mae = mean_absolute_error(target_test, target_test_pred)
print(f"Modell: {model} -- Test -- Mean Absolute Error: {mae:5.2f}")

<p><font color='black' size="5">
Aufbau AnalysewÃ¼rfel
</font></p>

In [ ]:
# Ãœbernahme der Testdaten
cube = DataFrame(data_test.copy())
cube.reset_index(inplace=True)

# Ãœbernahem Target real & predict
cube["real"] = DataFrame(target_test.values, columns=["real"])
cube["predict"] = DataFrame(target_test_pred, columns=["predict"])

<p><font color='black' size="5">
Visualisierung real vs predict
</font></p>

In [ ]:
# Boxplot
title_ = "Boxplot real vs predict"
px.box(cube[["real", "predict"]], title=title_, width=600, height=600)

In [ ]:
# Histogramm
title_ = "Histogramm Prices real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=10, text_auto=".2s", title=title_)
fig.update_layout(barmode="group", bargap=0.2, width=800, height=600)
fig.show()

<p><font color='black' size="5">
Fehlerhafte Vorhersagen
</font></p>

In [ ]:
cube["abs_Abw%"] = abs((cube["real"] - cube["predict"]) / cube["real"] * 100)
cube.head(10).style.format("{:,.1f}")

In [ ]:
cube.describe().T

In [ ]:
# Histogramm
title_ = "Histogramm absolute Abweichung"
fig = px.histogram(cube, x=["abs_Abw%"], nbins=20, text_auto=".2s", title=title_)
fig.update_layout(barmode="group", bargap=0.2, width=800, height=600)
fig.show()

<p><font color='black' size="5">
Residuals Plot
</font></p>

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Plot-Funtion, fÃ¼r Keras</font> </br></p>
def erstelle_residuals_plot(target_train, target_train_pred, target_test, target_test_pred, title):
    """Erstellt Residuals Plot mit Train/Test Split"""
    # Arrays zu 1D konvertieren
    target_train = np.array(target_train).ravel()
    target_train_pred = np.array(target_train_pred).ravel()
    target_test = np.array(target_test).ravel()
    target_test_pred = np.array(target_test_pred).ravel()

    df_train = DataFrame({
        'Vorhersage': target_train_pred,
        'Residuen': target_train_pred - target_train,
        'Set': 'Train'
    })

    df_test = DataFrame({
        'Vorhersage': target_test_pred,
        'Residuen': target_test_pred - target_test,
        'Set': 'Test'
    })

    df = concat([df_train, df_test])

    fig = px.scatter(df, x='Vorhersage', y='Residuen',
                    color='Set',
                    color_discrete_map={'Train': 'blue', 'Test': 'green'},
                    opacity=0.5,  # Transparenz hinzugefÃ¼gt
                    title=title,
                    labels={'Vorhersage': 'Vorhergesagte Werte',
                           'Residuen': 'Residuen'},
                    marginal_y='histogram',
                    width=1000,   # Breite in Pixeln
                    height=600)

    fig.add_hline(y=0, line_color="black")
    fig.update_layout(height=600, margin=dict(t=100))
    fig.update_layout(
        xaxis=dict(domain=[0, 0.85]),  # Scatter nimmt 85% der Breite ein
        xaxis2=dict(domain=[0.86, 1]),
        title='HÃ¤ufigkeit'
        )

    return fig

In [ ]:
erstelle_residuals_plot(target_train, target_train_pred, target_test, target_test_pred, "Residuals Plot Model keras")

# 5 | Deploy
---

<p><font color='black' size="5">ðŸ“‹ Checkliste</font></p>

âœ… Modellexport und -speicherung</br>
âœ… AbhÃ¤ngigkeiten und Umgebung</br>
âœ… Sicherheit und Datenschutz</br>
âœ… In die Produktion integrieren</br>
âœ… Tests und Validierung</br>
âœ… Dokumentation & Wartung</br>